In [1]:
%load_ext autoreload
%autoreload 2



In [2]:
# from transformer_mil import Transformer
import pdb
import torch
import os

In [7]:
os.listdir('/Users/awxlong/Desktop/my-studies/temp_data/CRC/Feature')

['resnet50',
 '.DS_Store',
 'resnet18',
 'prov-gigapath',
 'ctranspath',
 'uni',
 '8472de58-9075-4534-b00b-3a87ba2158da.TCGA-AD-6963-01Z-00-DX1.7df2e133-5f24-4c0a-b7f5-5a65fe3420c9.svs.pt',
 'tcga_folder_1.TCGA-A8-A085-01Z-00-DX1.2B52D1B8-5AD4-4BD6-ADF7-9D65B8EE2622.svs.pt',
 'tcga_folder_2.TCGA-A8-A084-01Z-00-DX1.2B52D1B8-5AD4-4BD6-ADF7-9D65B8EE2621.svs.pt',
 'a72573a6-2462-4185-b4e8-a081c95d8fd7.TCGA-AA-3713-01Z-00-DX1.8148ACEB-7C1E-4D29-B908-F3729657EA4F.svs.pt']

In [13]:
rand_vec = torch.rand(15, 1024)


In [14]:
rand_vec.shape
torch.save(rand_vec, \
           '/Users/awxlong/Desktop/my-studies/temp_data/CRC/Feature/uni/a72573a6-2462-4185-b4e8-a081c95d8fd7.TCGA-AA-3713-01Z-00-DX1.8148ACEB-7C1E-4D29-B908-F3729657EA4F.svs.pt')

In [31]:
embedding = torch.load('/Users/awxlong/Desktop/my-studies/temp_data/CRC/Feature/resnet50/8472de58-9075-4534-b00b-3a87ba2158da.TCGA-AD-6963-01Z-00-DX1.7df2e133-5f24-4c0a-b7f5-5a65fe3420c9.svs.pt')
embedding = torch.load('/Users/awxlong/Desktop/my-studies/temp_data/CRC/Feature/uni/8472de58-9075-4534-b00b-3a87ba2158da.TCGA-AD-6963-01Z-00-DX1.7df2e133-5f24-4c0a-b7f5-5a65fe3420c9.svs.pt')
embedding = torch.load('/Users/awxlong/Desktop/my-studies/temp_data/CRC/Feature/uni/a72573a6-2462-4185-b4e8-a081c95d8fd7.TCGA-AA-3713-01Z-00-DX1.8148ACEB-7C1E-4D29-B908-F3729657EA4F.svs.pt')
# embedding = torch.load('/Users/awxlong/Desktop/my-studies/temp_data/CRC/Feature/uni/tcga_folder_1.TCGA-A8-A085-01Z-00-DX1.2B52D1B8-5AD4-4BD6-ADF7-9D65B8EE2622_uni.svs.pt')

# embedding = torch.load('/Users/awxlong/Desktop/my-studies/temp_data/CRC/Feature/uni/tcga_folder_2.TCGA-A8-A084-01Z-00-DX1.2B52D1B8-5AD4-4BD6-ADF7-9D65B8EE2621.svs.pt')
embedding = torch.load('/Users/awxlong/Desktop/my-studies/temp_data/CRC/Feature/uni/tcga_folder_1.TCGA-A8-A085-01Z-00-DX1.2B52D1B8-5AD4-4BD6-ADF7-9D65B8EE2622.svs.pt')



In [32]:
embedding.shape

torch.Size([15518, 1536])

In [ ]:
model_config = {'heads': 8, 
                'dim_head': 64, 
                'dim': 512, 
                'mlp_dim': 512, 
                'input_dim':768}

In [ ]:
model = Transformer(num_classes=1, **model_config)
state_dict = torch.load('MSI_high_CRC_model.pth')
# Remove the criterion.pos_weight since it's not part of model
state_dict.pop('criterion.pos_weight', None)
# Remove the 'model.' prefix from the keys
new_state_dict = {}
for k, v in state_dict.items():
    new_key = k.replace('model.', '')
    new_state_dict[new_key] = v


# # Get the keys from both dictionaries
# pretrained_keys = set(new_state_dict.keys())
# model_keys = set(model.state_dict().keys())

# # Find keys that are in the pretrained weights but not in the model
# extra_keys = pretrained_keys - model_keys

# # Find keys that are in the model but not in the pretrained weights
# missing_keys = model_keys - pretrained_keys

# # Print the results
# print("Keys in pretrained weights but not in model:")
# for key in extra_keys:
#     print(f"  {key}")

# print("\nKeys in model but not in pretrained weights:")
# for key in missing_keys:
#     print(f"  {key}")

# # If you want to see the corresponding shapes of the tensors:
# print("\nShapes of extra keys in pretrained weights:")
# for key in extra_keys:
#     print(f"  {key}: {state_dict[key].shape}")

# print("\nShapes of missing keys in model:")
# for key in missing_keys:
#     print(f"  {key}: {model.state_dict()[key].shape}")
# pos_weight = new_state_dict['criterion.pos_weight'].item()
# print(f"Positive class weight used in training: {pos_weight}")
# pdb.set_trace()
# Load the modified state dictionary into your model
model.load_state_dict(new_state_dict)
# model.load_state_dict(torch.load('MSI_high_CRC_model.pth'), strict=True)
# pdb.set_trace()

In [ ]:
model

In [ ]:
# Load the state dict
state_dict = torch.load('path_to_your_pretrained_weights.pth')

# Remove the criterion.pos_weight
state_dict.pop('criterion.pos_weight', None)

# Get the model's state dict
model_state_dict = model.state_dict()

# Filter out the keys with size mismatches
filtered_state_dict = {k: v for k, v in state_dict.items() if k in model_state_dict and v.size() == model_state_dict[k].size()}

# Load the filtered state dict into your model
model.load_state_dict(filtered_state_dict, strict=False)

# Manually initialize the layers with size mismatches
torch.nn.init.xavier_uniform_(model.projection[0].weight)
torch.nn.init.zeros_(model.mlp_head[1].weight)
torch.nn.init.zeros_(model.mlp_head[1].bias)
